In [16]:
import heapq
import jieba
import jieba.analyse
import pandas as pd
from text2vec import Similarity
import datetime

In [29]:
# 停用詞與關鍵詞
stop = open("./chinese-similarity-master/stop_words.txt", "r+", encoding="utf-8")
stopword = stop.read().split("\n")
key = open("./chinese-similarity-master/traveldict.txt", "r+", encoding="utf-8")
keyword = key.read().split("\n")

# 檔案路徑
filePath = "./0_10000.csv"

# 檔案中的評論欄位
index = "綜合評論"

# 已知評論
def init_data(filePath, index):
    dream_data = pd.read_csv(filePath)
    return dream_data.values[:, index]

# 過濾停用詞，並使用自訂關鍵詞典
def strip_word(seg):
    # 讀取關鍵詞典
    jieba.load_userdict("./chinese-similarity-master/traveldict.txt")
    
    wordlist = []
    
    # 獲得關鍵字
    keywords = jieba.analyse.extract_tags(seg, topK=5, withWeight=False, allowPOS=("n"))
    
    # 遍歷分詞表
    for key in jieba.cut(seg):
        
        # 去除停用詞，去除單字且不在關鍵詞典，去除重複詞
        if (
            not (key.strip() in stopword)
            and (len(key.strip()) > 1 or key.strip() in keyword)
            and not (key.strip() in wordlist)
        ):
            wordlist.append(key)
            # print(key)

    # 停用詞去除 END
    stop.close()
    
    return "".join(wordlist)

# 將 str_arr 中的項目提前進行 strip_word 處理並儲存到字典中
def preprocess_str_arr(str_arr):
    preprocessed_dict = {}
    for item in str_arr:
        preprocessed_item = strip_word(item)
        preprocessed_dict[item] = preprocessed_item
    print(preprocessed_dict)
    return preprocessed_dict


# 利用 text2vec詞向量模型計算出兩段處理後的文本相似度
def similarity_calculation(preprocessed_dict, str_2):
    sim = Similarity()
    str_2 = strip_word(str_2)
    result = []
    for item, preprocessed_item in preprocessed_dict.items():
        result.append(sim.get_score(preprocessed_item, str_2))
    return result


# 將新用戶輸入訊息，轉換為關鍵詞文本(可以同時處理多個用戶輸入)
def deal_init_data(text_data):
    text_arr = []
    for item in text_data:
        # 關鍵字提取
        text_arr.append(strip_word(item))
    key_words = pd.DataFrame(text_arr, columns=["key_text"])
    
    # 將新用戶輸入訊息存成 csv
    key_words.to_csv("./chinese-similarity-master/dream_keywords.csv", sep=",", header=True, index=True,encoding = 'utf-8-sig')
    return key_words

In [ ]:
if __name__ == "__main__":
    
    # 開始測量所花時間
    startime = datetime.datetime.now()
    
    # 讀取已知文本 : init_data(filePath, index)
    key_arr = init_data("./chinese-similarity-master/base_content.csv", 1)
    
    # 過濾停用詞，並使用自訂關鍵詞典(預處理 key_arr 中的項目並儲存結果)
    preprocessed_dict = preprocess_str_arr(key_arr)
    
    # 結束測量
    endtime = datetime.datetime.now()
    
    # 輸出結果
    print("執行時間：", endtime - startime)

In [31]:
if __name__ == "__main__":
    
    # 開始測量所花時間
    startime = datetime.datetime.now()
    
    # 新用戶輸入
    text_data = ['浴室不乾淨','CP值高']
    
    for index, item in enumerate(demo_arr):
        
        result = similarity_calculation(preprocessed_dict, item)
        
        # 找到相似度最高的前 50 個
        re1 = map(result.index, heapq.nlargest(50, result))
        re2 = heapq.nlargest(50, result)
        print("原文本", item)
        for i, val in enumerate(list(re1)):
            print(i + 1, "對比结果：", key_arr[val], "，相似度：", re2[i])
            
    # 結束測量
    endtime = datetime.datetime.now()
    
    # 輸出結果
    print("執行時間：", endtime - startime)

2023-07-13 17:08:13.489 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


原文本 浴室不乾淨
1 對比结果： 服務人員態度很親切 ，相似度： 0.9207941293716431
2 對比结果： 過年期間價格親民停車方便且櫃檯人員也很親切門鎖是喇叭鎖廁所不是乾濕分離的 ，相似度： 0.8971003293991089
3 對比结果： 景觀很好空間大小合適空間大小沒有 ，相似度： 0.8702318072319031
4 對比结果： 房間浴缸有點小污漬 ，相似度： 0.8700262308120728
5 對比结果： 一樓空間大房間空間也很足夠洗衣機和烘衣機有點貴 ，相似度： 0.8688586950302124
6 對比结果： 地理位置好 ，相似度： 0.8604753017425537
7 對比结果： 可以 ，相似度： 0.8586379289627075
8 對比结果： 現煮早餐好好吃 ，相似度： 0.8574897646903992
9 對比结果： 服務親切無 ，相似度： 0.8539023399353027
10 對比结果： 舒適自在這次攜家帶眷入住讓我們一家一夜好眠到天亮讓人流連忘返 ，相似度： 0.8459216356277466
11 對比结果： 要不是過年期間不好訂房應該不會在選擇再次入住離市區近早餐樣式少不怎麼樣烤麵包機小烤箱都很髒床單有毛髮有?漬 ，相似度： 0.8363733291625977
12 對比结果： 早餐可以換連鎖早餐店的餐券環境鬧中取靜對女性稍不友好沒化妝桌備品 ，相似度： 0.8336490392684937
13 對比结果： 性價比高方案不理想下次直接飯店訂房 ，相似度： 0.8326670527458191
14 對比结果： 一定要再光臨窗外風景很棒房間非常乾淨 ，相似度： 0.8297973871231079
15 對比结果： 性價比高隔音不理想 ，相似度： 0.8245733976364136
16 對比结果： 房間雖大但硬體設備卻老舊停車位過少房間寬敞飯店停車位太少而且服務人員只會要入住者自己想辦法去附近找停車位 ，相似度： 0.8240305185317993
17 對比结果： 櫃檯服務員一位嬌小年輕的女服務員處理方式我就非常餐廳的服務員也非常熱忱像是朋友來自家這樣招呼房務人員態度也很好曾入住幾次這次的櫃檯服務員一位綁馬尾中性跟一位年長的兩位女服務員處理事情的態度跟圓融性讓我感覺非常差勁這

2023-07-13 17:18:07.051 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


原文本 CP值高
1 對比结果： 失望毛巾 ，相似度： 0.6288759708404541
2 對比结果： 失望毛巾 ，相似度： 0.6288759708404541
3 對比结果： 失望毛巾 ，相似度： 0.6288759708404541
4 對比结果： 離市區很近逛街購物方便 ，相似度： 0.5932097434997559
5 對比结果： 工作人員 ，相似度： 0.5477691292762756
6 對比结果： 早餐很棒 ，相似度： 0.5424857139587402
7 對比结果： 這一次來花蓮旅遊入住非凡假期飯店二晚很愉悅早餐簡單中西式都有新鮮可口非常滿意魯肉飯炒米苔目很美味房間舒適乾淨又安靜地點就在市區中心逛街用餐宵夜都很方便有免費自行車可以借用車況保養很好交誼廳咖啡現煮且自由取用櫃檯員工服務態度和藹親切是性價比很高的旅店下次來花蓮會再入住 ，相似度： 0.5351763963699341
8 對比结果： 房間內蚊子四五十隻蚊子才是房客噁心一個房間蚊子四五十隻是要房客怎麼睡衛生品質這麼差還好意思收費這麼貴噁心 ，相似度： 0.5349054336547852
9 對比结果： 竟然提供晚上的便當跟早上的早餐便當真是令我驚訝呀 ，相似度： 0.5307992696762085
10 對比结果： 櫃檯人員 ，相似度： 0.5247135162353516
11 對比结果： 早餐不佳 ，相似度： 0.5222870111465454
12 對比结果： 位置佳浴室水量大有貓浴室沒有可以放乾淨衣物的地方床太硬個人認知床上有螞蟻不少隻沙發有污漬穿鞋的長椅我進房前已斷裂沒早餐 ，相似度： 0.5170308351516724
13 對比结果： 雖然是老飯店設備跟裝潢都沒有新興旅店那麼吸睛但整體而言住起來很舒服床很好睡服務也都很到位有需要計程車可以請櫃檯幫忙叫到達飯店時如果車停在飯店門口工作人員還會幫忙開車門覺得受寵若驚健身房的重訓器材應該比較少人使用所以不是很齊備沒辦法練齊一個部位不過跑步機滑步機跟腳踏車都有適合想作作有氧的人房間的非常弱幾乎連不上健身房有很多啞鈴但竟然沒有羅馬椅 ，相似度： 0.5153748393058777
14 對比结果： 室內燈光昏暗總能聽見隔壁房間的聊天聲 ，相似度： 0.5149503350257874
15 對比结果： 隔音好差位

In [25]:
if __name__ == "__main__":
    
    # 開始測量所花時間
    startime = datetime.datetime.now()
    
    # 讀取已知文本 : init_data(filePath, index)
    key_arr = init_data("./chinese-similarity-master/base_content.csv", 1)
    
    # 讀取新用戶輸入
    # demo_arr = init_data("./chinese-similarity-master/demo.csv", 0)
    
    # 讀取新用戶輸入(經關鍵字提取)
    demo_arr = init_data("./chinese-similarity-master/dream_keywords.csv", 1)

    for index, item in enumerate(demo_arr):
        result = similarity_calculation(key_arr, item)
        
        # 找到相似度最高的前十個
        re1 = map(result.index, heapq.nlargest(10, result))
        re2 = heapq.nlargest(10, result)
        print("原文本", item)
        for i, val in enumerate(list(re1)):
            print(i + 1, "對比结果：", key_arr[val], "，相似度：", re2[i])
            
    # 結束測量
    endtime = datetime.datetime.now()
    
    # 輸出結果
    print("執行時間：", endtime - startime)

2023-07-13 13:50:45.034 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


原文本 環境乾淨隔音不好
1 對比结果： 隔音不好 ，相似度： 0.8287917375564575
2 對比结果： 房間很舒服隔音有點不好 ，相似度： 0.8249245882034302
3 對比结果： 隔音不錯 ，相似度： 0.7992836236953735
4 對比结果： 人員服務態度很好隔音效果有點差 ，相似度： 0.7654012441635132
5 對比结果： 都很棒空調及房間整潔度隔音稍差 ，相似度： 0.764006495475769
6 對比结果： 位置很清幽附近很安靜隔音不是很好 ，相似度： 0.7493597269058228
7 對比结果： 早餐可以再加強整潔隔音不太好 ，相似度： 0.7351045608520508
8 對比结果： 相當不錯的環境以及優秀的工作人員小缺點是冷氣的運轉聲影響睡眠 ，相似度： 0.7154563665390015
9 對比结果： 環境非常乾淨整潔 ，相似度： 0.7099565863609314
10 對比结果： 房間乾淨沒有異味 ，相似度： 0.7073772549629211


2023-07-13 13:53:55.390 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


原文本 服務態度
1 對比结果： 服務態度很好 ，相似度： 0.9999999403953552
2 對比结果： 服務態度很好 ，相似度： 0.9999999403953552
3 對比结果： 服務態度很好 ，相似度： 0.9999999403953552
4 對比结果： 服務態度很好 ，相似度： 0.9999999403953552
5 對比结果： 服務態度很好 ，相似度： 0.9999999403953552
6 對比结果： 服務人員態度好 ，相似度： 0.8534069061279297
7 對比结果： 服務態度滿分 ，相似度： 0.8388381004333496
8 對比结果： 服務到位無 ，相似度： 0.8388108015060425
9 對比结果： 員工的服務態度很好 ，相似度： 0.8375146389007568
10 對比结果： 員工的服務態度很好 ，相似度： 0.8375146389007568
執行時間： 0:06:19.307003
